# 1. 单步RNN

In [1]:
import tensorflow as tf
import numpy as np




### 1. basic cell

In [2]:
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=128) # state_size = 128
print(cell.state_size) # 128


128


In [3]:
inputs = tf.placeholder(np.float32, shape=(32, 100))# 32 是 batch_size


In [6]:
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态，形状为(batch_size, state_size)
output, h1 = cell.__call__(inputs, h0)

### 2. LSTM cell---两个隐藏层 h ,c 

In [9]:
lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=128)
inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = lstm_cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态
output, h1 = lstm_cell.__call__(inputs, h0)

print(h1.h)  # shape=(32, 128)
print(h1.c)  # shape=(32, 128)


Tensor("basic_lstm_cell/Mul_2:0", shape=(32, 128), dtype=float32)
Tensor("basic_lstm_cell/Add_1:0", shape=(32, 128), dtype=float32)


# 2. 执行多步：tf.nn.dynamic_rnn

In [10]:
X = np.random.randn(2, 10, 8)


In [12]:
X[1,6:]=0

In [14]:
X_lengths= [10,6]

In [17]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=64, state_is_tuple=True)

outputs, last_states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float64,
    sequence_length=X_lengths,
    inputs=X)

result = tf.contrib.learn.run_n(
    {"outputs": outputs, "last_states": last_states},
    n=1,
    feed_dict=None)

print(result[0])

assert result[0]["outputs"].shape == (2, 10, 64)

# 第二个example中的outputs超过6步(7-10步)的值应该为0
assert (result[0]["outputs"][1,7,:] == np.zeros(cell.output_size)).all()


Instructions for updating:
graph_actions.py will be deleted. Use tf.train.* utilities instead. You can use learn/estimators/estimator.py as an example.
Instructions for updating:
graph_actions.py will be deleted. Use tf.train.* utilities instead. You can use learn/estimators/estimator.py as an example.
Instructions for updating:
graph_actions.py will be deleted. Use tf.train.* utilities instead. You can use learn/estimators/estimator.py as an example.
{'outputs': array([[[-0.08384192,  0.06183881,  0.01755905, ...,  0.02954254,
         -0.04551525, -0.04980299],
        [-0.13860482,  0.15091965,  0.03361808, ..., -0.03315255,
         -0.09747536,  0.02441158],
        [-0.17416995,  0.0804932 ,  0.05496859, ...,  0.08393534,
          0.00220577, -0.05753896],
        ...,
        [-0.1237643 , -0.07398791, -0.03995043, ...,  0.08149298,
         -0.1735362 , -0.08055957],
        [-0.10806652, -0.05192893,  0.01843831, ...,  0.0865122 ,
         -0.23490919, -0.05260242],
        [

# 3. 堆叠RNNCell：MultiRNNCell

In [18]:

# 每调用一次这个函数就返回一个BasicRNNCell
def get_a_cell():
    return tf.nn.rnn_cell.BasicRNNCell(num_units=128)
# 用tf.nn.rnn_cell MultiRNNCell创建3层RNN
cell = tf.nn.rnn_cell.MultiRNNCell([get_a_cell() for _ in range(3)]) # 3层RNN
# 得到的cell实际也是RNNCell的子类
# 它的state_size是(128, 128, 128)
# (128, 128, 128)并不是128x128x128的意思
# 而是表示共有3个隐层状态，每个隐层状态的大小为128
print(cell.state_size) # (128, 128, 128)
# 使用对应的call函数
inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态
output, h1 = cell.call(inputs, h0)
print(h1) # tuple中含有3个32x128的向量

(128, 128, 128)
(<tf.Tensor 'cell_0/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'cell_1/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>, <tf.Tensor 'cell_2/basic_rnn_cell/Tanh:0' shape=(32, 128) dtype=float32>)
